# MIT BIH heart beat classification model for heartbeats


The libary used to collect features from raw ECG data is the neurokit.p library which is used in this project to find the locations of important parts of each ecg wave including r-peak, p-peak,q-peak,r-onset,p-onset,p-offset,q-onset,q-offset. Being as this library does not come with python it must be installed to use this model.

In [ ]:
pip install neurokit.py

In [11]:
import numpy as np
import pandas as pd
import csv
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn import tree
%run Functions.ipynb
%run FeaturesFunctions.ipynb
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

loaded functions


First we need to create a training file with all of the annotated files in order. the files range from 100-235 not inclusive, the file 113 does not work with the Neurokit library so it was excluded from the model training.
File name: 1 csv for feature extraction

In [12]:

AnnotatedFileNames=[*range(100,125)]+[*range(200,235)]
remove=[110,113,120,204,206,211,216,218,224,225,226,227,229]
for i in remove:
    AnnotatedFileNames.remove(i)
Training = pd.DataFrame(columns = ['File','Result', 'Sample#'] )

for i in AnnotatedFileNames:
    temp= pd.DataFrame(columns = Training.columns)

    FileName ="mitbih_database/" + str(i)+'annotations.txt'
    annotations = mlbeats.df_from_txt(FileName)
    temp['Sample#']=annotations['Sample#']
    temp['Result']=annotations['Result']
    temp['File']= i
    
    Training = pd.concat([Training, temp], axis=0)
    

In [22]:
print(len(Training))
Training.head()

92462


,index,File,Result,Sample#,beat_number,ECG_R_Peaks,ECG_Q_Peaks,ECG_S_Peaks,ECG_P_Peaks,ECG_P_Onsets,ECG_P_Offsets,ECG_R_Onsets
2,2,100,N,370,2,370.0,359.0,472.0,309.0,294.0,338.0,350.0
3,3,100,N,662,3,662.0,652.0,757.0,603.0,586.0,610.0,644.0
4,4,100,N,946,4,946.0,936.0,958.0,884.0,863.0,890.0,925.0
5,5,100,N,1231,5,1231.0,1221.0,1323.0,1168.0,1141.0,1202.0,1213.0
6,6,100,N,1515,6,1515.0,1505.0,1614.0,1462.0,1458.0,1476.0,1495.0


Adding neurokit data and aux data to training file.ipynbThen using the nerokit library we can add feature locations to set features in the data, Filename: Adding neurokit data and aux data to training file.ipynb

In [14]:
testFeatures=pd.DataFrame()
for file in AnnotatedFileNames:
    DATA= pd.DataFrame(np.nan, index=range(1), columns=testFeatures.columns)
    testFeatures=pd.concat([testFeatures,DATA])
    testFeatures=pd.concat([testFeatures,Bryson.NerokitExtraction(file)])


extracting File 100


C:\Users\bryso\anaconda3\lib\site-packages\neurokit2\epochs\eventrelated_utils.py:81: NeuroKitWarning: Input does not have an `ECG_Rate` column. Will skip all rate-related features.
  warn(
C:\Users\bryso\anaconda3\lib\site-packages\neurokit2\ecg\ecg_eventrelated.py:147: NeuroKitWarning: Input does not have an `ECG_Phase_Artrial` or `ECG_Phase_Ventricular` column. Will not indicate whether event onset concurs with cardiac phase.
  warn(
C:\Users\bryso\anaconda3\lib\site-packages\neurokit2\ecg\ecg_eventrelated.py:170: NeuroKitWarning: Input does not have an `ECG_Quality` column. Quality of the signal is not computed.
  warn(
C:\Users\bryso\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\bryso\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


extraction complete
extracting File 101


C:\Users\bryso\anaconda3\lib\site-packages\neurokit2\epochs\eventrelated_utils.py:81: NeuroKitWarning: Input does not have an `ECG_Rate` column. Will skip all rate-related features.
  warn(
C:\Users\bryso\anaconda3\lib\site-packages\neurokit2\ecg\ecg_eventrelated.py:147: NeuroKitWarning: Input does not have an `ECG_Phase_Artrial` or `ECG_Phase_Ventricular` column. Will not indicate whether event onset concurs with cardiac phase.
  warn(
C:\Users\bryso\anaconda3\lib\site-packages\neurokit2\ecg\ecg_eventrelated.py:170: NeuroKitWarning: Input does not have an `ECG_Quality` column. Quality of the signal is not computed.
  warn(
C:\Users\bryso\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\bryso\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


extraction complete
extracting File 102


...

extraction complete
extracting File 233


C:\Users\bryso\anaconda3\lib\site-packages\neurokit2\epochs\eventrelated_utils.py:81: NeuroKitWarning: Input does not have an `ECG_Rate` column. Will skip all rate-related features.
  warn(
C:\Users\bryso\anaconda3\lib\site-packages\neurokit2\ecg\ecg_eventrelated.py:147: NeuroKitWarning: Input does not have an `ECG_Phase_Artrial` or `ECG_Phase_Ventricular` column. Will not indicate whether event onset concurs with cardiac phase.
  warn(
C:\Users\bryso\anaconda3\lib\site-packages\neurokit2\ecg\ecg_eventrelated.py:170: NeuroKitWarning: Input does not have an `ECG_Quality` column. Quality of the signal is not computed.
  warn(
C:\Users\bryso\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\bryso\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


extraction complete
extracting File 234


C:\Users\bryso\anaconda3\lib\site-packages\neurokit2\epochs\eventrelated_utils.py:81: NeuroKitWarning: Input does not have an `ECG_Rate` column. Will skip all rate-related features.
  warn(
C:\Users\bryso\anaconda3\lib\site-packages\neurokit2\ecg\ecg_eventrelated.py:147: NeuroKitWarning: Input does not have an `ECG_Phase_Artrial` or `ECG_Phase_Ventricular` column. Will not indicate whether event onset concurs with cardiac phase.
  warn(
C:\Users\bryso\anaconda3\lib\site-packages\neurokit2\ecg\ecg_eventrelated.py:170: NeuroKitWarning: Input does not have an `ECG_Quality` column. Quality of the signal is not computed.
  warn(


extraction complete


We must then add the test features to the previously created training file.

In [15]:
#NewTraining=pd.concat([NewTraining,NewFeatures],axis=1)
Training = Training.reset_index(drop=True)
testFeatures=testFeatures.reset_index(drop=True)
Training=pd.concat([Training,testFeatures],axis=1)
Training = Training.reset_index()
Training.head()

,index,File,Result,Sample#,beat_number,ECG_R_Peaks,ECG_Q_Peaks,ECG_S_Peaks,ECG_P_Peaks,ECG_P_Onsets,ECG_P_Offsets,ECG_T_Peaks,ECG_T_Onsets,ECG_T_Offsets,ECG_R_Onsets,ECG_R_Offsets
0,0,100,+,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,100,N,77,1,77.0,66.0,88.0,NaN,NaN,NaN,144.0,NaN,147.0,NaN,91.0
2,2,100,N,370,2,370.0,359.0,472.0,309.0,294.0,338.0,432.0,418.0,468.0,350.0,387.0
3,3,100,N,662,3,662.0,652.0,757.0,603.0,586.0,610.0,783.0,761.0,791.0,644.0,676.0
4,4,100,N,946,4,946.0,936.0,958.0,884.0,863.0,890.0,1064.0,1045.0,1070.0,925.0,961.0


Due to the large amount of NaN values in the ECG_T_Peaks, ECG_T_Onsets ,ECG_T_Offsets ,and ECG_R_Offsets columns they are removed from the data.

In [16]:

Training = Training.drop(["ECG_T_Peaks","ECG_T_Onsets","ECG_T_Offsets","ECG_R_Offsets"],axis='columns')
print(len(Training))
Training.dropna(inplace=True)
print(len(Training))

110851
92462


Now, Interval data is collected from the time data to get useful information to build the model on.

In [17]:
Interval = pd.DataFrame(columns=["File","Abnormal","Result","R-R Interval","R Height","R_Onset-Rpeak","Q-Q Interval","Q Height","P_Onset-P_Offset","P Height","P-P Interval","P_Onset-Ppeak"])
RRLIST=np.array([])
QQLIST=np.array([])
R_Height=np.array([])
Q_Height=np.array([]) 
P_Height=np.array([])
Ron_RP=np.array([])
Pon_PP=np.array([])
Pon_Poff=np.array([])
PPLIST=np.array([])
Abnormal=np.array([])
Result=np.array([])
FILES=np.array([])
y=2
FileNumber=Training["File"][y]
RAWFile= pd.read_csv("mitbih_database/"+str(FileNumber)+".csv")



In [18]:


for i in Training["index"][:-1]:
    if (i-y>1):
        y=i
        continue
    if Training["Result"][i]=='N':
        Abnormal=np.append(Abnormal,0)
    else:
        Abnormal=np.append(Abnormal,1)
    if (Training["File"][i] != FileNumber):
        print("Reading File",FileNumber)
        FileNumber=Training["File"][i]
        RAWFile= pd.read_csv("mitbih_database/"+str(FileNumber)+".csv")
        
    RRLIST=np.append(RRLIST,Training["ECG_R_Peaks"][i]-Training["ECG_R_Peaks"][y])
    QQLIST=np.append(QQLIST,Training["ECG_Q_Peaks"][i]-Training["ECG_Q_Peaks"][y])
    PPLIST=np.append(PPLIST,Training["ECG_P_Peaks"][i]-Training["ECG_P_Peaks"][y])
    R_Height=np.append(R_Height,RAWFile[RAWFile.columns[1]][Training["ECG_R_Peaks"][i]])#
    Q_Height=np.append(Q_Height,RAWFile[RAWFile.columns[1]][Training["ECG_Q_Peaks"][i]])#
    P_Height=np.append(P_Height,RAWFile[RAWFile.columns[1]][Training["ECG_P_Peaks"][i]])#
    Ron_RP=np.append(Ron_RP,Training["ECG_R_Peaks"][i]-Training["ECG_R_Onsets"][i])
    Pon_PP=np.append(Pon_PP,Training["ECG_P_Peaks"][i]-Training["ECG_P_Onsets"][i])
    Pon_Poff=np.append(Pon_Poff,Training["ECG_P_Offsets"][i]-Training["ECG_P_Onsets"][i])
    Result=np.append(Result,Training["Result"][i])
    FILES=np.append(FILES,FileNumber)
    
    y=i

Interval["R-R Interval"]=RRLIST
Interval["Q-Q Interval"]=QQLIST
Interval["P-P Interval"]=PPLIST
Interval["R_Onset-Rpeak"]=Ron_RP
Interval["P_Onset-Ppeak"]=Pon_PP
Interval["P_Onset-P_Offset"]=Pon_Poff
Interval["R Height"]=R_Height#
Interval["Q Height"]=Q_Height#
Interval["P Height"]=P_Height#
Interval["File"]=FILES
Interval["Abnormal"]=Abnormal
Interval["Result"]=Result

print("Length of DataFrame",len(Interval))
Interval.head(10)





Reading File 100
Reading File 101
Reading File 102
Reading File 103
Reading File 104
Reading File 105
Reading File 106
Reading File 107
Reading File 108
Reading File 109
Reading File 111
Reading File 112
Reading File 114
Reading File 115
Reading File 116
Reading File 117
Reading File 118
Reading File 119
Reading File 121
Reading File 122
Reading File 123
Reading File 124
Reading File 200
Reading File 201
Reading File 202
Reading File 203
Reading File 205
Reading File 207
Reading File 208
Reading File 209
Reading File 210
Reading File 212
Reading File 213
Reading File 214
Reading File 215
Reading File 217
Reading File 219
Reading File 220
Reading File 221
Reading File 222
Reading File 223
Reading File 228
Reading File 230
Reading File 231
Reading File 232
Reading File 233
Length of DataFrame 83283


,File,Abnormal,Result,R-R Interval,R Height,R_Onset-Rpeak,Q-Q Interval,Q Height,P_Onset-P_Offset,P Height,P-P Interval,P_Onset-Ppeak
0,100.0,0.0,N,0.0,1212.0,20.0,0.0,922.0,44.0,983.0,0.0,15.0
1,100.0,0.0,N,292.0,1201.0,18.0,293.0,920.0,24.0,981.0,294.0,17.0
2,100.0,0.0,N,284.0,1186.0,21.0,284.0,895.0,27.0,975.0,281.0,21.0
3,100.0,0.0,N,285.0,1188.0,18.0,285.0,921.0,61.0,973.0,284.0,27.0
4,100.0,0.0,N,284.0,1201.0,20.0,284.0,915.0,18.0,974.0,294.0,4.0
5,100.0,0.0,N,294.0,1213.0,19.0,294.0,920.0,27.0,984.0,284.0,20.0
6,100.0,1.0,A,235.0,1193.0,19.0,235.0,914.0,42.0,978.0,238.0,23.0
7,100.0,0.0,N,358.0,1193.0,20.0,357.0,920.0,30.0,972.0,354.0,19.0
8,100.0,0.0,N,304.0,1202.0,20.0,304.0,918.0,25.0,975.0,304.0,18.0
9,100.0,0.0,N,292.0,1209.0,19.0,293.0,919.0,23.0,981.0,295.0,17.0


Now a decision Tree classification model is created to descern between normal and abrnormal heartbeats, the model is then scored using regular scoring techniques along with the AUC score 

In [19]:
#creatin
model1 = tree.DecisionTreeClassifier()

inputs = Interval.drop(["Result","Abnormal"],axis='columns')
Target = Interval[["Abnormal"]]



x_train1, x_test1, y_train1, y_test1 = train_test_split(inputs, Target,test_size=0.3)

model1.fit(X=x_train1,y=y_train1)
print(model1.score(x_test1,y_test1),"Regular")
print(roc_auc_score(y_test1, model1.predict(x_test1)),"AUC1\n")





0.9631378827296377 Regular
0.9542624683426774 AUC1



In [20]:
#With sampling rate of 1000
#0.9683361313187838 Regular
#0.9562155004639009 AUC1

#sampling rate of   360 
#0.9631378827296377 Regular
#0.9542624683426774 AUC1